In [1]:
import os
import pandas as pd
import numpy as np

# Set TF_GPU_ALLOCATOR environment variable
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import save_model
from tensorflow.keras.models import load_model
from keras_tuner import RandomSearch
from keras_tuner import HyperParameters
from sklearn.preprocessing import StandardScaler
import joblib

import warnings
# Suppress all warnings
warnings.filterwarnings("ignore")

In [2]:
# Load back X_train, X_test, y_train, and y_test
X_train = np.loadtxt('train_data/X_train_t10.txt')
X_test = np.loadtxt('train_data/X_test_t10.txt')
y_train = np.loadtxt('train_data/y_train_t10.txt')
y_test = np.loadtxt('train_data/y_test_t10.txt')

# Load the scaler for data normalization
scaler_file = 'models/scaler.pkl'
scaler = joblib.load(scaler_file)
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


CNN for Feature Extraction:

- Start with a CNN to extract features from the power measurements at different frequencies. Since the frequency information is spatially related, we can treat it as an image where each frequency bin is a pixel. Design the CNN architecture to have convolutional layers to capture local patterns in the frequency domain. The output of the CNN will be a feature map capturing important frequency-domain features.

RNN for Temporal Modeling:
- Feed the output of the CNN into an RNN (such as LSTM or GRU) to model temporal dependencies in the time-series data.
- The RNN will capture the temporal dynamics and sequential patterns in the power measurements over time.

This architecture allows the model to learn both frequency-domain features and temporal dependencies simultaneously.

Hybrid CNN-RNN Architecture:
- We can design a hybrid architecture where the CNN and RNN parts are connected sequentially. The CNN part processes the input frequency data to extract features, and the output is then fed into the RNN for temporal modeling.
- This architecture allows for both spatial (frequency) and temporal information to be captured effectively.

In [3]:
# Reshape the data to 3D for CNN input (samples, time steps, features)
X_train_reshaped = X_train_scaled.reshape(X_train_scaled.shape[0], X_train_scaled.shape[1], 1)
X_test_reshaped = X_test_scaled.reshape(X_test_scaled.shape[0], X_test_scaled.shape[1], 1)
y_train_reshaped = y_train.reshape((-1, 1))
y_test_reshaped = y_test.reshape((-1, 1))

I am building a hybrid CNN and RNN model, taking power measurement of RF activity from 0 to 6.0 GHz to determine whether a camera is being turned on or off, based on its electromagnetic emanation. I have the raw data splitted accoring to these variables: X_train, X_test, y_trarin, y_test are numpy array of dimension (1637, 76802), (410, 76802), (1637,), (410,) respectively, where the row is capture data taken every seconds, and columns are power measurement in dBm (watts) from 0 to 6.0 GHz frequency taken by spectrum analyzer


In [1]:
from tensorflow import keras

def build_model(hp):
    model = keras.Sequential()

    # For example, if your data is reshaped to (frequency_bins, time_steps, 1)
    input_shape = (X_train_reshaped.shape[1], 1, 1)
    
    # Add convolutional layers
    model.add(keras.layers.Conv2D(32, (3, 1), activation='relu', input_shape=input_shape))
    model.add(keras.layers.MaxPooling2D((2, 1)))  # Reduce only in one dimension
    model.add(keras.layers.Conv2D(64, (3, 1), activation='relu'))
    model.add(keras.layers.MaxPooling2D((2, 1)))  # Reduce only in one dimension
    model.add(keras.layers.Conv2D(128, (3, 1), activation='relu'))
    model.add(keras.layers.MaxPooling2D((2, 1)))  # Reduce only in one dimension

    # Flatten the output and add dense layers
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(128, activation='relu'))
    model.add(keras.layers.Dense(64, activation='relu'))
    model.add(keras.layers.Dropout(0.5))
    model.add(keras.layers.Dense(32, activation='relu'))
    model.add(keras.layers.Dense(1, activation='sigmoid'))

    # Compile the model
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    return model

Define a keras tuner for random search

In [5]:
# Define the Keras Tuner RandomSearch
tuner = RandomSearch( build_model,
                      objective='val_accuracy',
                      max_trials=10,
                      executions_per_trial=1,
                      directory='keras_tuning_10',
                      project_name='cnn_rnn_tuner')

# Perform hyperparameter search
tuner.search(X_train_reshaped, y_train_reshaped,
             epochs=10,
             validation_data=(X_test_reshaped, y_test_reshaped))

Trial 1 Complete [00h 00m 09s]
val_accuracy: 1.0

Best val_accuracy So Far: 1.0
Total elapsed time: 00h 00m 09s


Get the best model

In [6]:
# Get the best model
best_model = tuner.get_best_models(num_models=1)[0]
# Save the Keras model to an HDF5 file
model_file_path = os.path.join('models', 'hybrid_cnn_rnn.h5')
best_model.save(model_file_path)

In [7]:
# Get best hyperparameters
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]

In [8]:
# Evaluate the best model
evaluation = best_model.evaluate(X_test_reshaped, y_test_reshaped)
# Print evaluation results
print(evaluation)

13/13 [==============================] - 0s 4ms/step - loss: 9.5177e-04 - accuracy: 1.0000
[0.0009517695871181786, 1.0]


In [9]:
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 6554, 1, 32)       128       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 3277, 1, 32)      0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 3275, 1, 64)       6208      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 1637, 1, 64)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 1635, 1, 128)      24704     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 817, 1, 128)      0

In [10]:
for layer in best_model.layers:
    print('Input Layers:')
    print(layer.input)
    print('Output Layers:')
    print(layer.output)

Input Layers:
KerasTensor(type_spec=TensorSpec(shape=(None, 6556, 1, 1), dtype=tf.float32, name='conv2d_input'), name='conv2d_input', description="created by layer 'conv2d_input'")
Output Layers:
KerasTensor(type_spec=TensorSpec(shape=(None, 6554, 1, 32), dtype=tf.float32, name=None), name='conv2d/Relu:0', description="created by layer 'conv2d'")
Input Layers:
KerasTensor(type_spec=TensorSpec(shape=(None, 6554, 1, 32), dtype=tf.float32, name=None), name='conv2d/Relu:0', description="created by layer 'conv2d'")
Output Layers:
KerasTensor(type_spec=TensorSpec(shape=(None, 3277, 1, 32), dtype=tf.float32, name=None), name='max_pooling2d/MaxPool:0', description="created by layer 'max_pooling2d'")
Input Layers:
KerasTensor(type_spec=TensorSpec(shape=(None, 3277, 1, 32), dtype=tf.float32, name=None), name='max_pooling2d/MaxPool:0', description="created by layer 'max_pooling2d'")
Output Layers:
KerasTensor(type_spec=TensorSpec(shape=(None, 3275, 1, 64), dtype=tf.float32, name=None), name='conv